In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext('local')
spark = SparkSession(sc)

In [18]:
df = spark.read.csv("time_between_events_test.csv", header=True)

In [19]:
df.toPandas()

,userId,sessionId,playlist_id,event_name,timestamp
0,1,1,A,start,1
1,1,1,None,another,2
2,1,1,A,stop,3
3,1,1,B,start,4
4,1,1,None,another,5
5,1,1,C,start,6
6,1,1,C,stop,7
7,1,1,C,start,8
8,1,1,C,stop,9


In [20]:

from pyspark.sql import Window
import time
from pyspark.sql.types import StringType
import pyspark.sql.functions as F
from pyspark.sql.types import TimestampType, LongType

In [21]:
#Window.partitionBy(partitioning_column).orderBy(ordering_column).rangeBetween(Window.unboundedPreceding,last_row)

In [22]:
df = df.filter(df['event_name'].isin(['start', 'stop']))

In [23]:
df.toPandas()

,userId,sessionId,playlist_id,event_name,timestamp
0,1,1,A,start,1
1,1,1,A,stop,3
2,1,1,B,start,4
3,1,1,C,start,6
4,1,1,C,stop,7
5,1,1,C,start,8
6,1,1,C,stop,9


In [24]:
win = Window.partitionBy(['userId', 'sessionId']).orderBy('timestamp')
df = df.withColumn("event_duration_end", F.lead('timestamp').over(win))
df.toPandas()

,userId,sessionId,playlist_id,event_name,timestamp,event_duration_end
0,1,1,A,start,1,3
1,1,1,A,stop,3,4
2,1,1,B,start,4,6
3,1,1,C,start,6,7
4,1,1,C,stop,7,8
5,1,1,C,start,8,9
6,1,1,C,stop,9,None


In [25]:
df = df.filter((F.col("event_name") == 'start') & (F.col('event_duration_end').isNotNull()))
df.toPandas()

,userId,sessionId,playlist_id,event_name,timestamp,event_duration_end
0,1,1,A,start,1,3
1,1,1,B,start,4,6
2,1,1,C,start,6,7
3,1,1,C,start,8,9


In [27]:
df = df.withColumn('playlist_duration', F.col('event_duration_end') - F.col('timestamp'))
df.toPandas()

,userId,sessionId,playlist_id,event_name,timestamp,event_duration_end,playlist_duration
0,1,1,A,start,1,3,2.0
1,1,1,B,start,4,6,2.0
2,1,1,C,start,6,7,1.0
3,1,1,C,start,8,9,1.0


In [29]:
df.groupBy(['userId', 'sessionId', 'playlist_id', 'event_name']).agg(F.sum('playlist_duration').alias('playlist_duration')).toPandas()

,userId,sessionId,playlist_id,event_name,playlist_duration
0,1,1,A,start,2.0
1,1,1,B,start,2.0
2,1,1,C,start,2.0
